In [1]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
from docx import Document

document = Document('./tax.docx')
print(f"document == {document}")

full_text = ''
for paragraph in document.paragraphs:
    full_text += f'{paragraph.text}\n'

document == <docx.document.Document object at 0x00000205188C2350>


In [3]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []

    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)

    return text_list


In [5]:
chunk_list = split_text(full_text, 1500)

In [6]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [7]:
import chromadb

chroma_client = chromadb.Client()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [13]:
collection_name = 'tax_collection'
# tax_collection = chroma_client.create_collection(collection_name)
chroma_client.delete_collection(collection_name)

In [9]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [14]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [15]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [16]:
print(len(id_list))
print(len(chunk_list))

104
104


In [17]:
tax_collection.add(documents=chunk_list, ids=id_list)

Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [18]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_doc = tax_collection.query(query_texts=query)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
retrieved_doc['documents'][0]

[[' 신청이 있는 경우로서 사업장 소재지를 납세지로 지정하는 것이 세무관리상 부적절하다고 인정되어 그 신청대로 납세지 지정을 하지 아니한 경우에는 국세청장 또는 관할 지방국세청장은 그 뜻을 납세의무자 또는 그 상속인, 납세관리인이나 납세조합에 서면으로 각각 통지하여야 한다.\n③ 제1항에서 규정한 납세지의 지정 사유가 소멸한 경우 국세청장 또는 관할 지방국세청장은 납세지의 지정을 취소하여야 한다.\n④ 제1항에 따른 납세지의 지정이 취소된 경우에도 그 취소 전에 한 소득세에 관한 신고, 신청, 청구, 납부, 그 밖의 행위의 효력에는 영향을 미치지 아니한다.\n[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 

In [29]:
%pip install openai

In [31]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)



In [38]:
response.choices[0].message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 소득세법에 규정된 과세표준 및 세율을 적용해야 합니다. 대한민국의 소득세는 과세표준에 따라 누진세율이 적용되며, 기본공제 및 소득공제를 통해 과세소득을 줄일 수 있습니다.\n\n기본적으로 소득세 계산은 다음 단계로 이루어집니다:\n\n1. **총급여액 확인**: 연봉 5천만 원에서 각종 비과세 소득이 있을 경우 이를 제외한 금액이 총급여액이 됩니다.\n\n2. **근로소득공제**: 소득에 따라 일정 금액을 공제하는 것으로, 연봉 구간에 따라 다르게 계산됩니다. \n   - 예를 들어, 5천만 원의 총급여라면, 특정 계산식에 따라 근로소득공제가 이루어집니다.\n\n3. **과세표준 결정**: 총급여액에서 근로소득공제를 뺀 금액이 과세표준이 됩니다. \n\n4. **세율 적용**: 과세표준에 대해 누진세율을 적용합니다. 대한민국의 소득세는 기본적으로 6%에서 시작해 과세표준이 높아짐에 따라 세율이 증가합니다. \n   - 예를 들어, 2023년 기준으로 과세표준 1,200만원 이하일 경우 6%, 그 다음 구간부터는 더 높은 세율이 적용됩니다.\n\n5. **세액공제 및 세액 감면**: 산출된 세액에서 인적공제, 특별세액공제 등을 통해 최종 산출세액이 결정됩니다.\n\n각 개인의 상황에 따라 다양한 공제항목이 적용될 수 있으므로, 정확한 세액 계산을 위해서는 국세청 홈택스의 소득세 계산기를 사용하거나 세무사와 상담하는 것이 좋습니다. \n\n대략적으로 공제나 감면 없이 단순 계산으로는, 근로소득공제와 기본공제를 감안하면 약 4백만 원에서 5백만 원 사이의 세금을 부담하게 될 수 있지만 정확한 금액은 공제 항목에 따라 크게 달라질 수 있습니다.'